In [1]:
### Virulance HMM Table Output Anaylsis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SearchIO
%matplotlib inline

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [2]:
#df = pd.read_table('vhmm_hmm_table.txt', header=None)

In [3]:
num_hit = 0
#now parse the output
hit_list = []
hit_ids = []
with open('vhmm_hmm_table.txt', 'r') as input:
    for qresult in SearchIO.parse(input, 'hmmer3-tab'):
        query_id = qresult.id  #sequence ID from fasta
        hits = qresult.hits
        hit_ids = hit_ids+qresult.hit_keys
        hit_list.append(hits)
        

In [4]:
hit_list[1][1]

Hit(id='3300000167.a:SI39nov09_120mDRAFT_c1000196_51', query_id='GTP_EFTU', 1 hsps)

In [5]:
len(hit_ids)

183431

In [6]:
JGI_ids = []
Scaffolds = []
for hit in hit_ids:
    z = hit.split(':')
    JGI_ids.append(z[0])
    Scaffolds.append(z[1])

In [7]:
len(JGI_ids)

183431

In [24]:
JGI_ids = [x.split('.', 2)[0] for x in JGI_ids]

In [25]:
from collections import defaultdict

d = defaultdict(list)
for key, value in zip(JGI_ids, Scaffolds):
    d[key].append(value)

In [47]:
my_dict = dict(d)
dict(my_dict.items()[4:5])

{'3300000192': ['SI60aug11_100mDRAFT_c1000573_10',
  'SI60aug11_100mDRAFT_c1001443_12',
  'SI60aug11_100mDRAFT_c1000573_13',
  'SI60aug11_100mDRAFT_c1001334_9',
  'SI60aug11_100mDRAFT_c1000294_6',
  'SI60aug11_100mDRAFT_c1000573_9',
  'SI60aug11_100mDRAFT_c1001107_9',
  'SI60aug11_100mDRAFT_c1000294_3',
  'SI60aug11_100mDRAFT_c1000573_9',
  'SI60aug11_100mDRAFT_c1000573_9']}

In [150]:
JGI_ids = list(map(int, JGI_ids))
my_dict2 = dict(zip(JGI_ids,Scaffolds))
dft = pd.DataFrame.from_dict(my_dict2)

ValueError: If using all scalar values, you must pass an index

In [66]:
uniq_JGI_ids = list(set(JGI_ids))
len(uniq_JGI_ids)

1760

In [35]:
from collections import Counter
c = Counter(Scaffolds)
c.most_common(10)

[('Draft_10000146_99', 12),
 ('Draft_10010688_10', 12),
 ('Draft_10001099_43', 12),
 ('Draft_10001190_42', 12),
 ('SRS018971_WUGC_scaffold_10797_26', 10),
 ('Draft_10001610_24', 10),
 ('SRS018975_WUGC_scaffold_47375_19', 10),
 ('SRS015899_WUGC_scaffold_35320_19', 10),
 ('Draft_10001648_26', 10),
 ('Draft_10002066_39', 10)]

In [36]:
uniq_Scaffold = list(set(Scaffolds))
len(uniq_Scaffold)

142746

In [54]:
y = {k:sum(1 for x in v if x) for k,v in my_dict.items()}
#sum(y.values())


q = sorted(y.items(), key=lambda x: x[1], reverse=True)
q[1:10]

[('2061766007', 5950),
 ('3300002641', 3104),
 ('3300001450', 2596),
 ('3300000117', 2324),
 ('3300000553', 2294),
 ('3300000116', 2236),
 ('3300001605', 2067),
 ('3300001460', 1999),
 ('3300002466', 1960)]

### Load Key Information

In [15]:
df = pd.read_table('Viral_Projects.txt')

In [19]:
df.head()

,Domain,Status,Study Name,Taxon OID,Genome Name,Habitat Type,Habitat (from GOLD),Depth,Viral Contig Count
0,*,P,Human microbial communities from the National ...,7000000203,Human tongue dorsum microbial communities from...,Host-associated(human),Human,NaN,115
1,*,P,Microbial and viral regulation of community ca...,3300005508,Marine microbial and viral communities from ox...,Marine,Marine,NaN,86
2,*,P,Permafrost soil microbial communities from the...,3300005994,Permafrost soil microbial communities from the...,Terrestrial(soil),Soil,0,2
3,*,P,Amended soil microbial communities from New Yo...,3300012988,Soil microbial communities amended with fresh ...,Terrestrial(soil),soil,0,113
4,*,P,Permafrost soil microbial communities from the...,3300006864,Permafrost soil microbial communities from the...,Terrestrial(soil),Soil,NaN,4


In [71]:
len(df)

4687

In [67]:
df2 = df[df['Taxon OID'].isin(uniq_JGI_ids)]

In [70]:
len(df2)

1633

In [92]:
## Find those not in list
overlap = list(df2['Taxon OID'])
uniq_JGI_ids = list(map(int, uniq_JGI_ids))
missing_numbers = list(set(uniq_JGI_ids) - set(overlap))

In [95]:
len(missing_numbers)

127

In [101]:
overlap = list(map(str, overlap))
filtered_dict = dict((k, my_dict[k]) for k in overlap if k in my_dict)
len(filtered_dict)

1633

In [103]:
y = {k:sum(1 for x in v if x) for k,v in filtered_dict.items()}
sum(y.values())

166616

In [122]:
test = [k for i,j in filtered_dict.items() for k in j if k]

In [114]:
c = Counter(Scaffolds)
c.most_common(10)

[('Draft_10000146_99', 12),
 ('Draft_10010688_10', 12),
 ('Draft_10001099_43', 12),
 ('Draft_10001190_42', 12),
 ('SRS018971_WUGC_scaffold_10797_26', 10),
 ('Draft_10001610_24', 10),
 ('SRS018975_WUGC_scaffold_47375_19', 10),
 ('SRS015899_WUGC_scaffold_35320_19', 10),
 ('Draft_10001648_26', 10),
 ('Draft_10002066_39', 10)]

In [137]:
keys=[]
for key, value in filtered_dict.iteritems():
    for item in value:
        if item == 'Draft_10000146_99':
            keys.append(key)
            
set(keys)

{'3300001594'}

In [120]:
[name for name, age in filtered_dict.items() if age == 'Draft_10000146_99']

[]

In [124]:
{k:v for k, v in filtered_dict.items() if v == 'SRS018971_WUGC_scaffold_10797_26'}

{}